<a href="https://colab.research.google.com/github/anuragsingh472002/bis-lab-code/blob/main/Anut_colony.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

class AntColony:
    def __init__(self, distances, n_ants, n_iterations, alpha=1, beta=2, evaporation_rate=0.5, pheromone_constant=100):
        self.distances = distances
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromone_constant = pheromone_constant

        self.n_cities = len(distances)
        self.pheromone = np.ones((self.n_cities, self.n_cities))

    def run(self):
        best_route = None
        best_distance = float('inf')

        for iteration in range(self.n_iterations):
            all_routes = []
            all_distances = []

            for ant in range(self.n_ants):
                route, distance = self._construct_solution()
                all_routes.append(route)
                all_distances.append(distance)

                if distance < best_distance:
                    best_route = route
                    best_distance = distance

            self._update_pheromone(all_routes, all_distances)

        return best_route, best_distance

    def _construct_solution(self):
        route = []
        visited = set()
        current_city = random.randint(0, self.n_cities - 1)
        route.append(current_city)
        visited.add(current_city)

        while len(visited) < self.n_cities:
            probabilities = self._calculate_probabilities(current_city, visited)
            next_city = self._select_next_city(probabilities)
            route.append(next_city)
            visited.add(next_city)
            current_city = next_city

        route.append(route[0])  # Return to start
        distance = self._calculate_route_distance(route)
        return route, distance

    def _calculate_probabilities(self, current_city, visited):
        probabilities = []
        pheromone = self.pheromone[current_city]
        distances = self.distances[current_city]

        for city in range(self.n_cities):
            if city in visited:
                probabilities.append(0)
            else:
                probabilities.append((pheromone[city] ** self.alpha) * ((1.0 / distances[city]) ** self.beta))

        total = sum(probabilities)
        probabilities = [p / total if total > 0 else 0 for p in probabilities]
        return probabilities

    def _select_next_city(self, probabilities):
        return np.random.choice(range(self.n_cities), p=probabilities)

    def _calculate_route_distance(self, route):
        distance = 0
        for i in range(len(route) - 1):
            distance += self.distances[route[i]][route[i + 1]]
        return distance

    def _update_pheromone(self, all_routes, all_distances):
        self.pheromone *= (1 - self.evaporation_rate)

        for route, distance in zip(all_routes, all_distances):
            for i in range(len(route) - 1):
                self.pheromone[route[i]][route[i + 1]] += self.pheromone_constant / distance
                self.pheromone[route[i + 1]][route[i]] += self.pheromone_constant / distance

if __name__ == "__main__":
    distances = np.array([
        [0, 2, 2, 5, 7],
        [2, 0, 4, 8, 2],
        [2, 4, 0, 1, 3],
        [5, 8, 1, 0, 2],
        [7, 2, 3, 2, 0]
    ])

    colony = AntColony(distances, n_ants=10, n_iterations=100, alpha=1, beta=2, evaporation_rate=0.5, pheromone_constant=100)
    best_route, best_distance = colony.run()

    print(f"Best route: {best_route}")
    print(f"Best distance: {best_distance}")


Best route: [1, 4, 3, 2, 0, 1]
Best distance: 9
